In [2]:
import cv2

In [4]:
# Tải bộ phát hiện khuôn mặt Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Mở webcam
cap = cv2.VideoCapture(0)

capture_flag = False
cropped_images = []
while True:
    # Đọc một khung hình từ webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi hình ảnh từ BGR (OpenCV mặc định) sang grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Vẽ bounding boxes xung quanh các khuôn mặt
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Hiển thị khung hình với bounding boxes
    cv2.imshow('Face Detection', frame)

    # Nhấn Space để chụp
    if cv2.waitKey(1) & 0xFF == ord(' '):
        capture_flag = True
    
    if capture_flag:
        if len(faces) == 0:
            print("Không có khuôn mặt!")
        else:
            for (x, y, w, h) in faces:
                cropped_images.append(frame[y:y+h, x:x+w])
            print(faces)
        break

# Giải phóng nguồn tài nguyên
cap.release()
cv2.destroyAllWindows()

max_face = []
for img in cropped_images:
    if len(img) > len(max_face):
        max_face = img
cv2.imshow('face', max_face)

cv2.waitKey(0)
cv2.destroyAllWindows()

[[171 167 197 197]]


In [3]:
from skimage import feature
import matplotlib.pyplot as plt

In [5]:
gray_face = cv2.cvtColor(max_face, cv2.COLOR_BGR2GRAY)
cv2.imshow('gray face', gray_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Tính toán HOG và hiển thị kết quả
hog_features, hog_image = feature.hog(gray_face, visualize=True)

# Hiển thị ảnh gốc và ảnh HOG
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.axis('off')
plt.title('Original Image')
plt.imshow(gray_, cmap='gray')

plt.subplot(1, 2, 2)
plt.axis('off')
plt.title('HOG Features')
plt.imshow(hog_image, cmap='gray')

plt.show()

In [13]:
hog_features

array([0.28357019, 0.04260522, 0.18130108, ..., 0.19517921, 0.08102537,
       0.02486873])

In [54]:
cap.release()
cv2.destroyAllWindows()

In [62]:
# LẤY DATA TỪ CAM

# Tải bộ phát hiện khuôn mặt Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Mở webcam
cap = cv2.VideoCapture(0)
count_frame = 0
capture_flag = False
cropped_images = []
i = 0
while True:
    # Đọc một khung hình từ webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi hình ảnh từ BGR (OpenCV mặc định) sang grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Vẽ bounding boxes xung quanh các khuôn mặt
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        if count_frame % 10 == 0:
            cv2.imwrite(f'./face-data-Truong/faceTruong{i}.jpg', frame[y:y+h, x:x+w])
            i += 1
        
    count_frame += 1
    # Hiển thị khung hình với bounding boxes
    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

    # nghi
    if i == 1000:
        break

# Giải phóng nguồn tài nguyên
cap.release()
cv2.destroyAllWindows()


In [35]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [9]:
import os

def create_hog_vector(dir, hog_vector, target, target_value):
    for item in os.listdir(dir):
        img_dir = os.path.join(dir, item)
        img = cv2.imread(img_dir)
        img_resize = cv2.resize(img, (200, 200)) # resize ảnh bằng nhau
        gray_face = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
        features = feature.hog(gray_face)
        hog_vector.append(np.ravel(features))
        target.append(target_value)

# Bắt đầu duyệt
hog_vector = []
target = []

Nhat_dir = './data/Nhat'
Truong_dir = './data/Truong'

create_hog_vector(Nhat_dir, hog_vector, target, 0)
create_hog_vector(Truong_dir, hog_vector, target, 1)

In [10]:
# Phân chia dữ liệu thành tập huấn luyện và tập còn lại
X_train_val, X_test, y_train_val, y_test = train_test_split(hog_vector, target, test_size=0.2, random_state=42, stratify=target)

# Phân chia tập còn lại thành tập validation và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

In [57]:
mlp = MLPClassifier(hidden_layer_sizes=(5,), max_iter=15, activation='relu', solver='adam', alpha=0.1, random_state=42)
mlp.fit(X_train, y_train)

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1, hidden_layer_sizes=(5,), max_iter=15, random_state=42)

In [60]:
y_probs = mlp.predict_proba(X_val)
result = []
for i, probs in enumerate(y_probs):
    if (((probs[0] > probs[1]) and (y_val[i] == 0)) or ((probs[0] < probs[1]) and (y_val[i] == 1))):
        result.append(1)
    else:
        result.append(0)
print(result.count(0))
print(result.count(1))

5
374


In [66]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(X_train, y_train)

RandomForestClassifier()

In [68]:
probabilities = RF.predict_proba(X_val)
probabilities

array([[0.07, 0.93],
       [1.  , 0.  ],
       [0.92, 0.08],
       [0.97, 0.03],
       [1.  , 0.  ],
       [0.03, 0.97],
       [0.93, 0.07],
       [0.01, 0.99],
       [0.12, 0.88],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.05, 0.95],
       [1.  , 0.  ],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.03, 0.97],
       [0.08, 0.92],
       [0.  , 1.  ],
       [0.97, 0.03],
       [0.07, 0.93],
       [1.  , 0.  ],
       [0.05, 0.95],
       [1.  , 0.  ],
       [0.1 , 0.9 ],
       [0.03, 0.97],
       [0.52, 0.48],
       [0.99, 0.01],
       [0.03, 0.97],
       [0.99, 0.01],
       [0.05, 0.95],
       [0.07, 0.93],
       [0.04, 0.96],
       [0.02, 0.98],
       [0.03, 0.97],
       [0.04, 0.96],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.04, 0.96],
       [0.98, 0.02],
       [0.74, 0.26],
       [0.02, 0.98],
       [0.09, 0.91],
       [0.97, 0.03],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.08, 0.92],
       [0.95,

In [67]:
y_pred = RF.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [58]:
from sklearn.metrics import accuracy_score
y_pred = mlp.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9868073878627969


In [19]:
print(result.count(1))
print(result.count(0))

379
0


In [ ]:
# Khởi tạo ImageDataGenerator với rescale
datagen = ImageDataGenerator(rescale=1./255)

# Tạo generator cho dữ liệu huấn luyện
train_generator = datagen.flow_from_directory(
    'path_to_training_data',  # Đường dẫn đến thư mục chứa dữ liệu huấn luyện
    target_size=(224, 224),   # Kích thước ảnh đầu vào
    batch_size=32,            # Kích thước của mỗi lô dữ liệu
    class_mode='categorical'  # Chế độ phân loại là 'categorical'
)

# Tạo generator cho dữ liệu kiểm tra
validation_generator = datagen.flow_from_directory(
    'path_to_validation_data',  # Đường dẫn đến thư mục chứa dữ liệu kiểm tra
    target_size=(224, 224),     # Kích thước ảnh đầu vào
    batch_size=32,              # Kích thước của mỗi lô dữ liệu
    class_mode='categorical'    # Chế độ phân loại là 'categorical'
)